# zip export

In [3]:
import zipfile
import os
import shutil

DATASET_ORDER = [
    "WLAN (WiFi) RSS database for fingerprinting positioning",
    "WiFi RSS measurements in Tampere University multi-building campus 2017",
    "Wi-Fi Fingerprinting dataset with multiple simultaneous interfaces",
    "UJIndoorLoc",
    "DSI_dataset",
    "Crowdsourced WiFi database and benchmark software for indoor positioning"
]

def flatten_folder(folder_path):
    """Eğer klasörde sadece tek bir alt klasör varsa, onun içeriğini yukarı taşır."""
    entries = os.listdir(folder_path)
    if len(entries) == 1:
        subfolder = os.path.join(folder_path, entries[0])
        if os.path.isdir(subfolder):
            for item in os.listdir(subfolder):
                src = os.path.join(subfolder, item)
                dst = os.path.join(folder_path, item)
                shutil.move(src, dst)
            shutil.rmtree(subfolder)
            print(f"   └─ Flattened folder structure inside '{folder_path}'")

def extract_datasets(outer_zip="datasets.zip", output_dir="data"):
    os.makedirs(output_dir, exist_ok=True)
    
    with zipfile.ZipFile(outer_zip, 'r') as outer:
        inner_files = [f for f in outer.namelist() if f.lower().endswith(".zip")]
        print(f"Found {len(inner_files)} inner zip files.")
        
        for idx, pattern in enumerate(DATASET_ORDER, start=1):
            match = next((f for f in inner_files if pattern.lower() in f.lower()), None)
            if not match:
                print(f"⚠️ Could not find dataset for pattern: {pattern}")
                continue
            
            target_folder = os.path.join(output_dir, str(idx))
            os.makedirs(target_folder, exist_ok=True)
            
            print(f"[{idx}] Extracting '{match}' → {target_folder}")
            with outer.open(match) as inner_zip_file:
                with zipfile.ZipFile(inner_zip_file) as inner_zip:
                    inner_zip.extractall(target_folder)
            
            # Flatten klasör yapısı
            flatten_folder(target_folder)
    
    print("✅ All datasets extracted and flattened successfully.")

if __name__ == "__main__":
    extract_datasets()


Found 6 inner zip files.
[1] Extracting 'datasets/WLAN (WiFi) RSS database for fingerprinting positioning.zip' → data\1
   └─ Flattened folder structure inside 'data\1'
[2] Extracting 'datasets/WiFi RSS measurements in Tampere University multi-building campus 2017 - Zenodo 5174851.zip' → data\2
   └─ Flattened folder structure inside 'data\2'
[3] Extracting 'datasets/Wi-Fi Fingerprinting dataset with multiple simultaneous interfaces.zip' → data\3
   └─ Flattened folder structure inside 'data\3'
[4] Extracting 'datasets/UJIndoorLoc.zip' → data\4
   └─ Flattened folder structure inside 'data\4'
[5] Extracting 'datasets/DSI_dataset.zip' → data\5
[6] Extracting 'datasets/Crowdsourced WiFi database and benchmark software for indoor positioning.zip' → data\6
✅ All datasets extracted and flattened successfully.


#### Converts RSS and coordinate  information from different WiFi positioning (fingerprinting) data sets to JSON format.

In [5]:
import os
import time
import sklearn
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import json


def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add




files_add=find_the_way("./data/4/",".csv")
files_add








for file in files_add :
    j_rssi = defaultdict(dict)
    j_coo = defaultdict(dict)
    rssis= pd.read_csv(file)#,header=None)
    cols=['LONGITUDE', 'LATITUDE', 'FLOOR', 'BUILDINGID', 'SPACEID','RELATIVEPOSITION', 'USERID', 'PHONEID', 'TIMESTAMP']
    lon=rssis['LONGITUDE'].values
    lat=rssis['LATITUDE'].values
    for col in cols:
        del rssis[col]
    for ii,i in enumerate(rssis.values):
        for jj, j in enumerate(i):
            if j!=100:
                j_rssi[int(ii)][int(jj)]=int(j)
                j_coo[ii]=[lon[ii],lat[ii]]
   
    with open(file.replace("csv","_rss.json"), 'w', encoding='utf-8') as f:
        json.dump(j_rssi, f, ensure_ascii=False, indent=4)

    with open(file.replace("csv","_coo.json"), 'w', encoding='utf-8') as f:
        json.dump(j_coo, f, ensure_ascii=False, indent=4)
    
    print(f"✅  {file}")

✅  ./data/4/trainingData.csv
✅  ./data/4/validationData.csv


#### Calculates the distances between different measurement points in WiFi positioning data (based on coordinates) and saves them to CSV files.

In [6]:
import json
import itertools
import math
import csv
from tqdm import tqdm
import os

In [7]:
def is_global_coordinate(coord):
    """
   Checks whether the coordinates are global.
   - If the latitude is between (-90, 90) and the longitude is between (-180, 180), they are global.
   - If both coordinates are above 90, they are considered local X/Y.
    """
    lat, lon = coord
    if -90 <= lat <= 90 and -180 <= lon <= 180:
        return True  # Küresel koordinat (lat/lon)
    if lat > 90 and lon > 90:
        return False  # Yerel koordinat (X/Y)
    
    # Şüpheli durumlarda uyarı ver
    print(f"UYARI: Koordinatlar tam tespit edilemedi, varsayılan olarak yerel kabul ediliyor! {coord}")
    return False  # Varsayılan olarak yerel kabul et


def haversine_distance(lat1, lon1, lat2, lon2):
    """Haversine formülü ile iki küresel koordinat arasındaki mesafeyi hesaplar (km cinsinden)."""
    R = 6371  # Dünya'nın yarıçapı (km)
    
    # Dereceden radyana çevirme
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Farklar
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formülü
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Sonuç km cinsinden döndürülür

def euclidean_distance(x1, y1, x2, y2):
    """Öklidyen mesafe hesaplar (yerel koordinatlar için)."""
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_distance(coord1, coord2):
    """
    Koordinatların türüne göre uygun mesafe hesaplama yöntemi seçer.
    - Eğer her iki koordinat da lat/lon ise Haversine
    - Eğer her ikisi de x/y ise Öklidyen
    - Karışık girişlerde None döndürerek işlemi atlar.
    """
    is_global_1 = is_global_coordinate(coord1)
    is_global_2 = is_global_coordinate(coord2)

    if is_global_1 and is_global_2:
        return haversine_distance(*coord1, *coord2), "Haversine"
    elif not is_global_1 and not is_global_2:
        return euclidean_distance(*coord1, *coord2), "Euclidean"
    else:
        return None, "Geçersiz"  # None döndürerek hatayı önle


def intersection_count(list1, list2):
    """İki listenin ortak eleman sayısını döndürür."""
    return len(set(list1) & set(list2))


In [8]:
prelist=find_the_way("./data/4",".json","")
files_add =  dict(zip(prelist[1::2], prelist[0::2]))
for i in files_add:
    print(i)

./data/4\trainingData._rss.json
./data/4\validationData._rss.json


In [9]:
for file in files_add:
    # JSON dosyalarını oku
    with open(file, "r") as f:
         rss= json.load(f)


    with open(files_add[file], "r") as f:
        data  = json.load(f)

        
    # Verileri uygun formata çevir (Anahtar: Nokta ID, Değer: (x, y) veya (lat, lon))
    points = {int(k): tuple(v) for k, v in data.items()}
    # CSV dosyasını aç ve verileri anlık olarak yaz
    with open(file.replace(".json","@.csv"), "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["id1", "id2", "estimated_distance", "method"])  # Başlık satırı
		


        total = len(points) * (len(points) - 1) // 2  # kombinasyon sayısı

        for (i, point1), (j, point2) in tqdm(itertools.combinations(points.items(), 2), total=total):

                list1 = list(rss[str(i)].keys())
                list2 = list(rss[str(j)].keys())
        
                if intersection_count(list1, list2):  # Ortak veri varsa mesafeyi hesapla
                    try:
                        distance = euclidean_distance(point1[0],point1[1],point2[0],point2[1])
                        writer.writerow([i, j, round(distance, 2), "euclidean_distance"])  # Dosyaya yaz
                    except ValueError as e:
                        print(f"Hata: {e}")
    print(f"{file}  kaydedildi.")


100%|████████████████████████████████████████████████████████████████| 197219730/197219730 [21:47<00:00, 150872.76it/s]


./data/4\trainingData._rss.json  kaydedildi.


100%|██████████████████████████████████████████████████████████████████████| 616605/616605 [00:04<00:00, 125229.15it/s]

./data/4\validationData._rss.json  kaydedildi.


####  It calculates various distance metrics (e.g. correlation, Euclidean, cosine, Jaccard) to measure the similarity or difference between WiFi fingerprint data, and records these as feature extraction in CSV files..

In [10]:
import csv
import numpy as np
import scipy.spatial.distance
from tqdm import tqdm

## 1.4 Feature Extraction (Intersection Based) - Simplified
def feature_extraction_file(data, name, fps):
    features = [["correlation",
                "chebyshev", 
                "intersecting_mac",
                "euclidean",
                "cosine",
                "jensenshannon",
                "jaccard",
                "canberra",
                "minkowski",
                "real"]]
    
    for i in tqdm(data, position=0, leave=True):
        fp1 = fps[i[0]]
        fp2 = fps[i[1]]
        feature = feature_extraction(fp1, fp2) 
        feature.append(i[2])
        features.append(feature)
    
    with open(name, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerows(features) 

def feature_extraction(fp1, fp2):
    # Kesişim (intersection) kullanarak ortak MAC adreslerini bul
    common_macs = set(fp1.keys()).intersection(set(fp2.keys()))
    
    # Eğer kesişim boşsa, tüm mesafeleri 0 olarak döndür
    if not common_macs:
        intersecting_mac = 0
        # Boş vektörler için mesafe hesaplamaları
        output_data = [0, 0, intersecting_mac, 0, 0, 0, 0, 0, 0]
        return output_data
    
    # Kesişen MAC sayısı
    intersecting_mac = len(common_macs)
    
    # Sadece ortak MAC'ler için vektörleri oluştur
    f1 = [fp1[mac] for mac in common_macs]
    f2 = [fp2[mac] for mac in common_macs]
    
    # Güvenli mesafe hesaplamaları
    def safe_distance(func, v1, v2, default_value=0):
        """Mesafe hesaplama fonksiyonunu güvenli şekilde çalıştırır"""
        try:
            result = func(v1, v2)
            if np.isnan(result) or np.isinf(result):
                return default_value
            return result
        except (ValueError, ZeroDivisionError, RuntimeWarning):
            return default_value
    
    # Vektörleri numpy array'e çevir
    f1_arr = np.array(f1, dtype=float)
    f2_arr = np.array(f2, dtype=float)
    
    # Özel durumları kontrol et
    f1_std = np.std(f1_arr)
    f2_std = np.std(f2_arr)
    
    # Mesafe hesaplamaları
    correlation = safe_distance(scipy.spatial.distance.correlation, f1_arr, f2_arr, 1.0)
    chebyshev = safe_distance(scipy.spatial.distance.chebyshev, f1_arr, f2_arr, 0)
    euclidean = safe_distance(scipy.spatial.distance.euclidean, f1_arr, f2_arr, 0)
    
    # Cosine için özel kontrol (sıfır vektör kontrolü)
    if f1_std == 0 and f2_std == 0:
        cosine = 0  # İki vektör de sabit ise mesafe sıfır
    elif f1_std == 0 or f2_std == 0:
        cosine = 1  # Bir vektör sabit ise maksimum mesafe
    else:
        cosine = safe_distance(scipy.spatial.distance.cosine, f1_arr, f2_arr, 1.0)
    
    jensenshannon = safe_distance(scipy.spatial.distance.jensenshannon, f1_arr, f2_arr, 0)
    canberra = safe_distance(scipy.spatial.distance.canberra, f1_arr, f2_arr, 0)
    minkowski = safe_distance(scipy.spatial.distance.minkowski, f1_arr, f2_arr, 0)
    
    # Jaccard mesafesi için binary vektörler (güvenli hesaplama)
    threshold = -70  
    f1_binary = [1 if x > threshold else 0 for x in f1]
    f2_binary = [1 if x > threshold else 0 for x in f2]
    
    # Jaccard için özel kontrol (tüm sıfır vektör kontrolü)
    if sum(f1_binary) == 0 and sum(f2_binary) == 0:
        jaccard = 0  # İki vektör de tüm sıfır ise benzer
    elif sum(f1_binary) == 0 or sum(f2_binary) == 0:
        jaccard = 1  # Bir vektör tüm sıfır ise maksimum mesafe
    else:
        jaccard = safe_distance(scipy.spatial.distance.jaccard, f1_binary, f2_binary, 1.0)
    
    output_data = [correlation,
                  chebyshev,
                  intersecting_mac,
                  euclidean,
                  cosine,
                  jensenshannon,
                  jaccard,
                  canberra,
                  minkowski]
    
    # NaN değerleri 0 ile değiştir
    output_data = [0 if x != x else x for x in output_data]
    
    return output_data

In [12]:
prelist=find_the_way("./data/4/","@.csv","")
prelist

['./data/4/validationData._rss@.csv']

In [13]:
for p in prelist:

    print(p)

    with open(p.replace("@.csv",".json")) as f:
        fps_train = json.load(f)
    
    with open(p) as f:
        train_data = []
        train_h = csv.DictReader(f)
        for pair in tqdm(train_h):
            train_data.append([pair['id1'],pair['id2'],float(pair['estimated_distance'])])
    print("Train Data loaded!!")
    feature_extraction_file(train_data,p.replace("@.csv","-distance.csv"),fps_train)
    print("\n\n")
    print("*"*100)

./data/4/validationData._rss@.csv


174416it [00:00, 179518.82it/s]


Train Data loaded!!


  0%|                                                                                       | 0/174416 [00:00<?, ?it/s]C:\Users\kahra\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\spatial\distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)
100%|████████████████████████████████████████████████████████████████████████| 174416/174416 [01:10<00:00, 2479.74it/s]





****************************************************************************************************


### Wifi Distance Estimation Model - XGB

In [27]:
#!/usr/bin/env python
# coding: utf-8

import warnings
warnings.filterwarnings("ignore")

import os
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

# Derin Öğrenme modeli için
from tensorflow.keras.models import load_model

def load_trained_model(model_name):
    """Eğitilmiş modeli yükler"""
    try:
        if model_name == "ANN":
            model_path = f"regression_models/{model_name}_model.h5"
            model = load_model(model_path)
        else:
            model_path = f"regression_models/{model_name}_model.pkl"
            with open(model_path, "rb") as f:
                model = pickle.load(f)
        print(f"✅ {model_name} modeli başarıyla yüklendi: {model_path}")
        return model
    except Exception as e:
        print(f"❌ Hata: {model_name} modeli yüklenemedi! - {e}")
        return None

def predict_and_save(model_name, data_path, output_folder="prediction_outputs"):
    """
    Veriyi yükler, model ile tahmin yapar ve sonuçları kaydeder
    
    Parametreler:
    - model_name: Kullanılacak model adı (örn: "BR", "XGB", "ANN")
    - data_path: Tahmin yapılacak CSV dosyasının yolu
    - output_folder: Sonuçların kaydedileceği klasör
    """
    
    # Çıktı klasörünü oluştur
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    print(f"\n{'='*50}")
    print(f"📊 Model: {model_name}")
    print(f"📁 Veri: {data_path}")
    print(f"{'='*50}\n")
    
    # 1. Modeli Yükle
    model = load_trained_model(model_name)
    if model is None:
        return
    
    # 2. Veriyi Yükle
    try:
        data_df = pd.read_csv(data_path)
        print(f"✅ Veri başarıyla yüklendi: {data_df.shape[0]} satır, {data_df.shape[1]} sütun")
    except Exception as e:
        print(f"❌ Hata: Veri yüklenemedi! - {e}")
        return
    
    # 3. Özellik çıkarımı (eğer 'real' sütunu varsa kaldır)
    if 'real' in data_df.columns:
        X_data = data_df.drop('real', axis=1)
        has_real_values = True
        real_values = data_df['real']
    else:
        X_data = data_df
        has_real_values = False
        real_values = None
    
    # 4. Tahmin Yap
    print("🔮 Tahmin yapılıyor...")
    try:
        if model_name == "ANN":
            predictions = model.predict(X_data).flatten()
        else:
            predictions = model.predict(X_data)
        print(f"✅ Tahmin tamamlandı! {len(predictions)} adet tahmin üretildi.")
    except Exception as e:
        print(f"❌ Hata: Tahmin yapılamadı! - {e}")
        return
    
    # 5. Sonuçları Hazırla
    result_df = pd.DataFrame()
    
    if has_real_values:
        result_df['real'] = real_values
        result_df['estimated'] = predictions
        result_df['error'] = result_df['real'] - result_df['estimated']
        result_df['absolute_error'] = np.abs(result_df['error'])
        
        # Hata istatistikleri
        mae = np.mean(result_df['absolute_error'])
        rmse = np.sqrt(np.mean(result_df['error']**2))
        print(f"\n📈 Performans Metrikleri:")
        print(f"   MAE (Ortalama Mutlak Hata): {mae:.4f}")
        print(f"   RMSE (Kök Ortalama Kare Hata): {rmse:.4f}")
    else:
        result_df['estimated'] = predictions
    
    # 6. Sonuçları Kaydet
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename =data_path.replace("._rss-distance.csv","_predictions.csv")
    
    result_df.to_csv(output_filename, index=False)
    print(f"\n✅ Sonuçlar kaydedildi: {output_filename}")
    print(f"\n{'='*50}\n")
    
    return result_df



In [30]:
files_add=find_the_way("./data/4/","rss-distance.csv")
files_add

['./data/4/trainingData._rss-distance.csv',
 './data/4/validationData._rss-distance.csv']

In [31]:
for i in files_add:
    folder_path = os.path.dirname(os.path.abspath(i))
    
    # Kullanılabilir modeller
    available_models = [ "XGB"]  # "DTR", "KNN", "LR", "ANN" ekleyebilirsiniz
    
    # Tahmin yapılacak veri dosyası
    data_file = i
    
    # Her model için tahmin yap
    for model_name in available_models:
        predict_and_save(
            model_name=model_name,
            data_path=data_file,
            output_folder=folder_path
        )
    
    print("\n🎉 Tüm tahminler tamamlandı!")


📊 Model: XGB
📁 Veri: ./data/4/trainingData._rss-distance.csv

✅ XGB modeli başarıyla yüklendi: regression_models/XGB_model.pkl
✅ Veri başarıyla yüklendi: 67588605 satır, 10 sütun
🔮 Tahmin yapılıyor...
✅ Tahmin tamamlandı! 67588605 adet tahmin üretildi.

📈 Performans Metrikleri:
   MAE (Ortalama Mutlak Hata): 35.7442
   RMSE (Kök Ortalama Kare Hata): 58.7216

✅ Sonuçlar kaydedildi: ./data/4/trainingData_predictions.csv



🎉 Tüm tahminler tamamlandı!

📊 Model: XGB
📁 Veri: ./data/4/validationData._rss-distance.csv

✅ XGB modeli başarıyla yüklendi: regression_models/XGB_model.pkl
✅ Veri başarıyla yüklendi: 174416 satır, 10 sütun
🔮 Tahmin yapılıyor...
✅ Tahmin tamamlandı! 174416 adet tahmin üretildi.

📈 Performans Metrikleri:
   MAE (Ortalama Mutlak Hata): 27.9444
   RMSE (Kök Ortalama Kare Hata): 46.1992

✅ Sonuçlar kaydedildi: ./data/4/validationData_predictions.csv



🎉 Tüm tahminler tamamlandı!


# Create and move distance files

In [38]:
real=pd.read_csv("./data/4/validationData._rss@.csv")
est=pd.read_csv("./data/4/validationData_predictions.csv")
real["estimated_distance"]=est["estimated"].values
real.to_csv("./data/4/validationData_WDE.csv", index=False)

In [39]:
real=pd.read_csv("./data/4/trainingData._rss@.csv")
est=pd.read_csv("./data/4/trainingData_predictions.csv")
real["estimated_distance"]=est["estimated"].values
real.to_csv("./data/4/trainingData_WDE.csv", index=False)

In [33]:
folders=["TrainingGeo", "TrainingWDE" , "ValidationGeo", "ValidationWDE"]

In [35]:
for folder_name in folders:
    try:
        os.mkdir(folder_name)
    except FileExistsError:
        pass

In [42]:
import shutil


copies=[['./data/4/trainingData.csv', "TrainingGeo/data.csv"],  ['./data/4/trainingData._rss@.csv', "TrainingGeo/data_distances.csv"],
['./data/4/validationData.csv', "ValidationGeo/data.csv"],  ['./data/4/validationData._rss@.csv', "ValidationGeo/data_distances.csv"],
['./data/4/trainingData.csv', "TrainingWDE/data.csv"],  ['./data/4/trainingData_WDE.csv', "TrainingWDE/data_distances.csv"],
['./data/4/validationData.csv', "ValidationWDE/data.csv"],  ['./data/4/validationData_WDE.csv', "ValidationWDE/data_distances.csv"]]




for c in copies:
    source = c[0]
    destination = c[1]
    shutil.copy2(source, destination)
    print(source, destination)

./data/4/trainingData.csv TrainingGeo/data.csv
./data/4/trainingData._rss@.csv TrainingGeo/data_distances.csv
./data/4/validationData.csv ValidationGeo/data.csv
./data/4/validationData._rss@.csv ValidationGeo/data_distances.csv
./data/4/trainingData.csv TrainingWDE/data.csv
./data/4/trainingData_WDE.csv TrainingWDE/data_distances.csv
./data/4/validationData.csv ValidationWDE/data.csv
./data/4/validationData_WDE.csv ValidationWDE/data_distances.csv
